In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

import numpy as np

from copy import deepcopy

# LeNet

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2),   # [28x28x6]
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),       # [14x14x6]
            nn.Conv2d(6, 16, kernel_size=5),             # [10x10x16]
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)        # [5x5x16]
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(5*5*16, 120),                      # [120]
            nn.Sigmoid(),
            nn.Linear(120, 84),                          # [84]
            nn.Sigmoid(),
            nn.Linear(84, 10)                            # [10]
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)        # [400]  
        x = self.fc_layers(x)
        x = F.log_softmax(x, dim=1)        
        return x

# learning_function()

In [15]:
def learning_function(model, train_loader, test_loader, 
                      epochs=10, learning_rate=0.001, device='auto'):
    
    metrics = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    
    # Определение устройства
    if device == 'auto':
        device_for_learning = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    elif device == 'cpu':
        device_for_learning = torch.device("cpu")
    elif device == 'gpu':
        device_for_learning = torch.device("gpu")
    else:
        raise ValueError("Invalid device choice. Use 'auto', 'cpu', or 'gpu'.")

    model.to(device_for_learning)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Не знаем что взять - пробуем Adam
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Планировщик скорости обучения

    # Цикл обучения
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for data, target in train_loader: #tqdm_notebook(train_loader, desc='Train', leave=False): 
            data, target = data.to(device_for_learning), target.to(device_for_learning)
            
            output = model(data)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        scheduler.step()
        
        # Валидация
        model.eval()
        correct = 0
        total = 0
        with torch.inference_mode():
            for data, target in test_loader: #tqdm_notebook(test_loader, desc='Train', leave=False): 
                data, target = data.to(device_for_learning), target.to(device_for_learning)
                logits = model(data)
                _, predicted = torch.max(logits.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f} | "
            f"LR: {scheduler.get_last_lr()[0]:.5f} | "
            f"Test Accuracy: {100 * correct / total:.2f}%")

    # Сохранение модели
    # torch.save(model.state_dict(), "lenet_fashionmnist.pth")

# Загрузка датасета, настройка параметров

In [4]:
# Гиперпараметры
BATCH_SIZE = 128
EPOCHS = 15
LEARNING_RATE = 0.005
torch.manual_seed(42)

# Трансформации для данных
transform = transforms.Compose([
    transforms.ToTensor() #,
    #transforms.Normalize((0.2860,), (0.3530,))  # Статистика FashionMNIST
])

### Реализация DS

In [5]:
# Загрузка FashionMNIST
train_dataset = datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root='./data',
    train=False,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [16]:
model = LeNet()
learning_function(model, train_loader, test_loader, epochs=15, learning_rate=0.005, device='auto')

Epoch [1/15] Loss: 0.9809 | LR: 0.00500 | Test Accuracy: 74.34%
Epoch [2/15] Loss: 0.5334 | LR: 0.00500 | Test Accuracy: 81.81%
Epoch [3/15] Loss: 0.4361 | LR: 0.00500 | Test Accuracy: 84.38%
Epoch [4/15] Loss: 0.3835 | LR: 0.00500 | Test Accuracy: 85.20%
Epoch [5/15] Loss: 0.3503 | LR: 0.00250 | Test Accuracy: 86.56%
Epoch [6/15] Loss: 0.3143 | LR: 0.00250 | Test Accuracy: 86.71%
Epoch [7/15] Loss: 0.3042 | LR: 0.00250 | Test Accuracy: 87.63%
Epoch [8/15] Loss: 0.2940 | LR: 0.00250 | Test Accuracy: 87.98%
Epoch [9/15] Loss: 0.2871 | LR: 0.00250 | Test Accuracy: 87.87%
Epoch [10/15] Loss: 0.2769 | LR: 0.00125 | Test Accuracy: 88.30%
Epoch [11/15] Loss: 0.2607 | LR: 0.00125 | Test Accuracy: 88.38%
Epoch [12/15] Loss: 0.2558 | LR: 0.00125 | Test Accuracy: 88.69%
Epoch [13/15] Loss: 0.2523 | LR: 0.00125 | Test Accuracy: 88.89%
Epoch [14/15] Loss: 0.2480 | LR: 0.00125 | Test Accuracy: 89.00%
Epoch [15/15] Loss: 0.2446 | LR: 0.00063 | Test Accuracy: 88.82%


In [14]:
model = LeNet()
learning_function(model, train_loader, test_loader, epochs=15, learning_rate=0.005, device='auto')

Epoch [1/15] Loss: 1.0908 | LR: 0.00500 | Test Accuracy: 75.12%
Epoch [2/15] Loss: 0.5515 | LR: 0.00500 | Test Accuracy: 80.30%
Epoch [3/15] Loss: 0.4490 | LR: 0.00500 | Test Accuracy: 82.29%
Epoch [4/15] Loss: 0.4033 | LR: 0.00500 | Test Accuracy: 84.48%
Epoch [5/15] Loss: 0.3757 | LR: 0.00250 | Test Accuracy: 85.52%
Epoch [6/15] Loss: 0.3344 | LR: 0.00250 | Test Accuracy: 86.18%
Epoch [7/15] Loss: 0.3230 | LR: 0.00250 | Test Accuracy: 86.78%
Epoch [8/15] Loss: 0.3130 | LR: 0.00250 | Test Accuracy: 87.04%
Epoch [9/15] Loss: 0.3033 | LR: 0.00250 | Test Accuracy: 87.02%
Epoch [10/15] Loss: 0.2965 | LR: 0.00125 | Test Accuracy: 87.10%
Epoch [11/15] Loss: 0.2782 | LR: 0.00125 | Test Accuracy: 87.91%
Epoch [12/15] Loss: 0.2738 | LR: 0.00125 | Test Accuracy: 88.23%
Epoch [13/15] Loss: 0.2690 | LR: 0.00125 | Test Accuracy: 88.20%
Epoch [14/15] Loss: 0.2648 | LR: 0.00125 | Test Accuracy: 87.63%
Epoch [15/15] Loss: 0.2617 | LR: 0.00063 | Test Accuracy: 87.67%


In [7]:
print(train_loader.dataset)
print(test_loader.dataset)
print(train_loader.dataset.data.shape, train_loader.dataset.data.sum())

# Сбор всех данных в один тензор
data = torch.cat([img for img, _ in train_loader.dataset], dim=0)

# Расчет среднего и std
mean = data.mean().item()
std = data.std().item()

print(f"Mean: {mean:.4f}, Std: {std:.4f}")

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )
torch.Size([60000, 28, 28]) tensor(3431114169)
Mean: 0.2860, Std: 0.3530


### Реализация HW

In [8]:
# Загрузка датасета
fashion_mnist_dataset = datasets.FashionMNIST(root="fashion",  
                                              train=True,
                                              download=True, 
                                              transform=transform)
# Содержит 60000 семплов и 10 классов
images = fashion_mnist_dataset.data.view([60000, 1, 28, 28]).float()

labels = deepcopy(fashion_mnist_dataset.targets)# - 1

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.01)

train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
batch_size = 128

train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_data, batch_size=len(val_data))

In [9]:
print(fashion_mnist_dataset)
print(fashion_mnist_dataset.data.shape, fashion_mnist_dataset.data.sum())

import numpy as np

# Сбор всех данных в один тензор
data = torch.cat([img for img, _ in fashion_mnist_dataset], dim=0)

# Расчет среднего и std
mean = data.mean().item()
std = data.std().item()

print(f"Mean: {mean:.4f}, Std: {std:.4f}")

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: fashion
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
torch.Size([60000, 28, 28]) tensor(3431114169)
Mean: 0.2860, Std: 0.3530


In [10]:
model2 = LeNet()
learning_function(model2, train_dl, val_dl, epochs=3, learning_rate=0.005, device='auto')

Train:   0%|          | 0/465 [00:00<?, ?it/s]

Train:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/3] Loss: 0.7897 | LR: 0.00500 | Test Accuracy: 78.50%


Train:   0%|          | 0/465 [00:00<?, ?it/s]

Train:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch [2/3] Loss: 0.4517 | LR: 0.00500 | Test Accuracy: 81.67%


Train:   0%|          | 0/465 [00:00<?, ?it/s]

Train:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch [3/3] Loss: 0.3839 | LR: 0.00500 | Test Accuracy: 86.33%


# Обучение моделей

In [11]:
#model = LeNet(how_inizialize='relu')
#learning_function(model, train_loader, test_loader, epochs=10, learning_rate=0.005, device='cpu')

In [12]:
#model = LeNet(how_inizialize='sigmoid')
#learning_function(model, train_loader, test_loader, epochs=10, learning_rate=0.005, device='auto')